In [8]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet',include_top = False)

Using TensorFlow backend.


In [9]:
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

In [10]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
print(type(loss))
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output

    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

<class 'tensorflow.python.ops.variables.RefVariable'>


In [12]:
dream = model.input

grads = K.gradients(loss,dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss,grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values


def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...', i, '번째 손실 :', loss_value)
        x += step * grad_values
    return x

In [14]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)


def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [15]:
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = './img.png'

img = preprocess_image(base_image_path)


original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)


successive_shapes = successive_shapes[::-1]


original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('처리할 이미지 크기', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='./final_dream.png')

처리할 이미지 크기 (245, 303)
... 0 번째 손실 : 1.920893
... 1 번째 손실 : 2.3215683
... 2 번째 손실 : 3.0540714
... 3 번째 손실 : 3.7989187
... 4 번째 손실 : 4.4901867
... 5 번째 손실 : 5.154647
... 6 번째 손실 : 5.804199
... 7 번째 손실 : 6.432978
... 8 번째 손실 : 7.0123963
... 9 번째 손실 : 7.631334
... 10 번째 손실 : 8.15699
... 11 번째 손실 : 8.707196
... 12 번째 손실 : 9.254874
... 13 번째 손실 : 9.753261
처리할 이미지 크기 (343, 424)
... 0 번째 손실 : 3.2722514
... 1 번째 손실 : 4.721453
... 2 번째 손실 : 5.906826
... 3 번째 손실 : 6.8921804
... 4 번째 손실 : 7.852454
... 5 번째 손실 : 8.714684
... 6 번째 손실 : 9.49358
처리할 이미지 크기 (481, 594)
... 0 번째 손실 : 3.2716565
... 1 번째 손실 : 4.547176
... 2 번째 손실 : 5.6744866
... 3 번째 손실 : 6.6939416
... 4 번째 손실 : 7.688407
... 5 번째 손실 : 8.710686
... 6 번째 손실 : 9.743526
